<a href="https://colab.research.google.com/github/adrianortega93/Paradigmas-de-Programacion/blob/main/ProyectoFinal_Ortega_Adrian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center; padding: 20px;">

  <h2 style="color: #2E86C1; margin-bottom: 5px;">Universidad Casa Grande</h2>
  <h3 style="color: #117864; margin-top: 0;">Maestría en Inteligencia Artificial y Ciencia de Datos</h3>

  <hr style="width: 60%; border: 1px solid #ccc; margin: 20px auto;">

  <h3 style="color: #884EA0; margin-bottom: 0;">Proyecto Final (Avanzado)</h3>
  <p style="font-size: 18px; margin-top: 5px;"><em>Simulación de un Pipeline de ML de Producción en un Entorno de Notebook</em></p>

  <p><strong>Autor:</strong> Adrian Ortega</p>
  <p><strong>Curso:</strong> Paradigmas de Programación para Inteligencia Artificial y Análisis de Datos</p>

</div>


<p>Este proyecto simula un flujo completo de <strong>Machine Learning</strong> en producción utilizando <strong>principios de ingeniería de software</strong> dentro de un entorno Colab. Se eligió un dataset complejo del dominio financiero con múltiples desafíos:</p>

<ul>
  <li>Alta cardinalidad en variables categóricas.</li>
  <li>Numerosos valores nulos en distintas columnas.</li>
  <li>Clases desbalanceadas (casos de incumplimiento escasos).</li>
  <li>Necesidad de un preprocesamiento diferenciado según tipo de dato.</li>
</ul>

<p>Se deja la ruta del archivo utilizado para este proyecto para una mejor visualización de su contenido y caracteristicas: <a href=" https://www.kaggle.com/datasets/mishra5001/credit-card">Credit Card Fraud Detection</a></p>

<p>El proyecto se estructura mediante módulos virtuales usando <code>%%writefile</code>, pruebas unitarias, logging, tipado estático y generación de requerimientos.</p>


<h2 style="color:#117A65">Configuración (config.py)</h2>

<p>En este módulo definimos todas las variables clave del pipeline:</p>

<ul>
  <li>URL y nombre del dataset.</li>
  <li>Columnas numéricas, categóricas, objetivo y columnas a eliminar.</li>
  <li>Semilla aleatoria para reproducibilidad.</li>
  <li>Parámetros del modelo y proporción del test set.</li>
</ul>

<p>Esta organización permite un mantenimiento limpio, evita "números mágicos" en el código y facilita modificar parámetros desde un único lugar.</p>


In [27]:
%%writefile config.py
# --------------------------------------------------------------------------------------------------------------
#                       Módulo de configuración para el proyecto de Machine Learning.
# --------------------------------------------------------------------------------------------------------------

import logging

# ------- Rutas y Nombres de Archivos -------
DATASET_URL = "mishra5001/credit-card"
DATASET_FILE_NAME = 'application_data.csv'
PIPELINE_NAME = 'trained_pipeline.joblib'

# ------- Variables del Dataset -------
TARGET_VARIABLE = 'TARGET'
FEATURES_TO_DROP = ['SK_ID_CURR']

# ------- Columnas numéricas, categóricas y de texto para el preprocesamiento. -------
NUMERIC_FEATURES = [
    'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
    'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS',
    'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG',
    'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
    'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG',
    'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
    'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
    'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
    'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
    'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
    'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI',
    'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
    'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
    'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
    'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
    'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_YEAR'
]

CATEGORICAL_FEATURES = [
    'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'WEEKDAY_APPR_PROCESS_START',
    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
    'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE',
    'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE',
    'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL'
]

# ------- Parámetros del Modelo y del Preprocesamiento -------
TEST_SIZE = 0.25
RANDOM_STATE = 42

MODEL_PARAMS = {
    'n_estimators': 100,
    'max_depth': 10,
    'random_state': RANDOM_STATE
}

# ------- Configuración del Logging -------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Overwriting config.py


<h2 style="color:#884EA0">Preprocesamiento (processing.py)</h2>

<p>El preprocesador se implementa como una clase que construye un <code>ColumnTransformer</code> complejo. Incluye:</p>

<ul>
  <li><strong>Numéricas</strong>: imputación con la mediana + escalado con <code>StandardScaler</code>.</li>
  <li><strong>Categóricas</strong>: imputación por moda + <code>OneHotEncoder</code> con <code>handle_unknown='ignore'</code>.</li>
  <li><strong>Desbalanceo</strong>: integración de <code>SMOTETomek</code>, que mejora el resampleo con <code>TomekLinks</code> para reducir ambigüedad.</li>
</ul>

<p>Este diseño modular permite aplicar técnicas adecuadas para cada tipo de dato en un flujo robusto y reutilizable.</p>


In [28]:
%%writefile processing.py
# --------------------------------------------------------------------------------------------------------------
#                                         Módulo de Procesamiento
# --------------------------------------------------------------------------------------------------------------

from typing import List
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

class MLPreprocessor:
    """
    Clase para construir un pipeline de preprocesamiento de datos modular y reutilizable.

    Encapsula la lógica de transformación para características numéricas y categóricas.
    """

    def __init__(self, numeric_features: List[str], categorical_features: List[str]):
        """
        Inicializa el preprocesador con listas de nombres de características.

        Args:
            numeric_features (List[str]): Lista de nombres de columnas numéricas.
            categorical_features (List[str]): Lista de nombres de columnas categóricas.
        """
        self.numeric_features = numeric_features
        self.categorical_features = categorical_features

    def get_column_transformer(self) -> ColumnTransformer:
        """
        Crea y retorna un ColumnTransformer para el preprocesamiento de datos.

        Este preprocesador aplica:
        - Imputación de mediana y escalado estándar a las características numéricas.
        - Imputación de moda y codificación One-Hot a las características categóricas.

        Returns:
            ColumnTransformer: El objeto ColumnTransformer configurado.
        """
        numeric_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])

        categorical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_pipeline, self.numeric_features),
                ('cat', categorical_pipeline, self.categorical_features)
            ],
            remainder='drop'
        )
        return preprocessor

    def get_full_pipeline(self, model) -> ImbPipeline:
        """
        Construye un pipeline completo que incluye preprocesamiento,
        manejo de desbalance de clases y un modelo clasificador.

        Args:
            model: El modelo clasificador de Scikit-learn a utilizar.

        Returns:
            ImbPipeline: El pipeline completo.
        """
        preprocessor = self.get_column_transformer()

        pipeline = ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('sampler', SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))),
            ('classifier', model)
        ])
        return pipeline

Overwriting processing.py


<h2 style="color:#B9770E">Entrenamiento y Evaluación (train.py)</h2>

<p>El script <code>train.py</code> ejecuta todo el flujo de entrenamiento:</p>

<ol>
  <li>Carga de datos desde Kaggle mediante <code>kagglehub</code>.</li>
  <li>División en train/test con <code>stratify</code>.</li>
  <li>Creación del pipeline con preprocesamiento + <code>RandomForestClassifier</code>.</li>
  <li>Entrenamiento y evaluación con <code>accuracy, f1, precision, recall</code>.</li>
  <li>Guardado del pipeline como <code>.joblib</code>.</li>
</ol>

<p>Además, se integró <strong>logging</strong> y <code>try...except</code> para registrar el flujo y manejar errores como <code>FileNotFoundError</code>.</p>


In [29]:
%%writefile train.py
# --------------------------------------------------------------------------------------------------------------
#                                         Módulo de Entrenamiento
# --------------------------------------------------------------------------------------------------------------

import os
import pandas as pd
import joblib
import kagglehub
import logging

from typing import Dict, Any
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.ensemble import RandomForestClassifier

from processing import MLPreprocessor
import config

logger = logging.getLogger(__name__)

def run_training() -> Dict[str, Any]:
    """
    Ejecuta el flujo completo de entrenamiento del modelo de Machine Learning.

    Incluye la carga de datos, la división, la construcción del pipeline,
    el entrenamiento, la evaluación y el guardado del modelo entrenado.

    Returns:
        Dict[str, Any]: Un diccionario con las métricas de rendimiento del modelo.
    """
    logger.info("--- INICIANDO ENTRENAMIENTO ---")

    # ------- 1. Carga y preparación de los datos -------
    logger.info("1. Cargando y dividiendo los datos...")
    try:
        path = kagglehub.dataset_download(config.DATASET_URL)
        csv_file_path = os.path.join(path, config.DATASET_FILE_NAME)

        # Se procede solo a tomar los primeros 10k de registros para efectos de prueba
        # ya que el archivo tiene mas de 300k y al ejecutarlo demora mucho
        #df = pd.read_csv(csv_file_path)
        df = pd.read_csv(csv_file_path, nrows=10000)
    except FileNotFoundError as e:
        logger.error(f"Error al cargar el archivo: {e}")
        raise e

    X = df.drop([config.TARGET_VARIABLE] + config.FEATURES_TO_DROP, axis=1)
    y = df[config.TARGET_VARIABLE]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=config.TEST_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=y
    )

    # ------- 2. Creación del pipeline -------
    logger.info("2. Creando el pipeline de preprocesamiento y modelo...")
    preprocessor = MLPreprocessor(
        numeric_features=config.NUMERIC_FEATURES,
        categorical_features=config.CATEGORICAL_FEATURES
    )
    model = RandomForestClassifier(**config.MODEL_PARAMS)
    full_pipeline = preprocessor.get_full_pipeline(model)

    # ------- 3. Entrenamiento del pipeline -------
    logger.info("3. Entrenando el pipeline...")
    full_pipeline.fit(X_train, y_train)

    # ------- 4. Evaluación del modelo -------
    logger.info("4. Evaluando el modelo...")
    predictions = full_pipeline.predict(X_test)

    metrics = {
        'accuracy': accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'),
        'precision': precision_score(y_test, predictions, average='weighted'),
        'recall': recall_score(y_test, predictions, average='weighted')
    }

    logger.info(f"Métricas de rendimiento:\n{classification_report(y_test, predictions)}")

    # ------- 5. Guardado del pipeline -------
    logger.info("5. Guardando el pipeline entrenado...")
    joblib.dump(full_pipeline, config.PIPELINE_NAME)
    logger.info(f"¡Entrenamiento finalizado y pipeline guardado en '{config.PIPELINE_NAME}'!")

    return metrics

if __name__ == '__main__':
    run_training()

Overwriting train.py


<h2 style="color:#566573">Predicción (predict.py)</h2>

<p>El módulo de predicción recibe una muestra como diccionario, carga el pipeline guardado y realiza la predicción.</p>

<p>Incluye:</p>

<ul>
  <li>Carga segura del modelo entrenado.</li>
  <li>Transformación del input a <code>DataFrame</code>.</li>
  <li>Predicción y cálculo de probabilidades.</li>
  <li>Formato interpretativo: <strong>"Pago"</strong> o <strong>"Incumplimiento"</strong>.</li>
</ul>

<p>El uso de <code>logging</code> y <code>warnings.filterwarnings</code> asegura trazabilidad y limpieza en la salida.</p>


In [30]:
%%writefile predict.py
# --------------------------------------------------------------------------------------------------------------
#                                         Módulo de Predicción
# --------------------------------------------------------------------------------------------------------------

from typing import Dict, Any
import joblib
import pandas as pd
import warnings
import logging

import config

logger = logging.getLogger(__name__)

# ------- Suprimimos las advertencias de versiones -------
warnings.filterwarnings("ignore")

def make_prediction(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Realiza una predicción utilizando un pipeline entrenado.

    Carga el pipeline guardado, prepara los datos de entrada y devuelve la
    predicción y las probabilidades asociadas.

    Args:
        input_data (Dict[str, Any]): Un diccionario con los datos de entrada de una sola muestra.

    Returns:
        Dict[str, Any]: Un diccionario con la predicción, las probabilidades o un mensaje de error.
    """
    logger.info("--- INICIANDO PREDICCIÓN ---")

    # ------- 1. Carga del pipeline entrenado -------
    try:
        pipeline = joblib.load(config.PIPELINE_NAME)
    except FileNotFoundError:
        logger.error(f"Error: El pipeline entrenado no se encontró en '{config.PIPELINE_NAME}'.")
        return {'error': 'El pipeline entrenado no se encontró. Por favor, ejecuta train.py primero.'}

    # ------- 2. Preparación de los datos de entrada -------
    input_df = pd.DataFrame([input_data])

    # ------- 3. Realizar la predicción -------
    prediction = pipeline.predict(input_df)
    probabilities = pipeline.predict_proba(input_df)

    # ------- 4. Formatear la salida -------
    prediction_label = 'Incumplimiento (Target=1)' if prediction[0] == 1 else 'Pago (Target=0)'

    result = {
        'prediction': prediction_label,
        'probability_pago': probabilities[0][0],
        'probability_incumplimiento': probabilities[0][1]
    }

    logger.info("Predicción realizada exitosamente.")
    logger.info(f"Resultado: {result}")

    return result

Overwriting predict.py


<h2 style="color:#A93226">Testing (test_processing.py)</h2>

<p>Se desarrollaron pruebas con <code>pytest</code> para asegurar el correcto funcionamiento del pipeline:</p>

<ul>
  <li><strong>test_pipeline_output_dimensionality</strong>: verifica que la salida tenga la cantidad esperada de columnas.</li>
  <li><strong>test_pipeline_handles_missing_values</strong>: comprueba que los valores nulos se imputen correctamente sin errores.</li>
</ul>

<p>Estas pruebas previenen errores en producción y aseguran que el pipeline sea tolerante a entradas reales.</p>


In [31]:
%%writefile test_processing.py
# --------------------------------------------------------------------------------------------------------------
#                                         Módulo de Pruebas
# --------------------------------------------------------------------------------------------------------------

import pytest
import pandas as pd
import numpy as np
from processing import MLPreprocessor
from sklearn.compose import ColumnTransformer

# Datos de prueba para simular el dataset
# ------- 4. Formatear la salida -------
@pytest.fixture
def sample_data():
    return pd.DataFrame({
        'numeric_feature_1': [10, 20, None, 40],
        'numeric_feature_2': [1.1, 2.2, 3.3, 4.4],
        'categorical_feature': ['A', 'B', 'A', 'B']
    })

def get_features():
    return ['numeric_feature_1', 'numeric_feature_2'], ['categorical_feature']

# ------- Prueba 1: Verificar que el pipeline produce la dimensionalidad correcta -------
def test_pipeline_output_dimensionality(sample_data):
    numeric_features, categorical_features = get_features()
    preprocessor = MLPreprocessor(numeric_features, categorical_features)
    transformer = preprocessor.get_column_transformer()

    X_transformed = transformer.fit_transform(sample_data)

    assert X_transformed.shape[1] == 4

# ------- Prueba 2: Verificar que el pipeline maneja y imputa valores nulos -------
def test_pipeline_handles_missing_values(sample_data):
    numeric_features, categorical_features = get_features()
    preprocessor = MLPreprocessor(numeric_features, categorical_features)
    transformer = preprocessor.get_column_transformer()

    transformer.fit(sample_data)

    X_transformed = transformer.transform(sample_data.iloc[2:3])

    # El test ahora verifica que el valor imputado NO sea NaN,
    # ya que el StandardScaler posterior cambia el valor numérico.
    assert not np.isnan(X_transformed[0, 0])

Overwriting test_processing.py


<h2 style="color:#2471A3">Orquestación</h2>

<p>En esta sección:</p>

<ol>
  <li>Se ejecuta <code>run_training()</code> para entrenar y guardar el modelo.</li>
  <li>Se selecciona una muestra real del dataset (una fila) y se realiza la predicción.</li>
  <li>Se muestra la predicción junto con la probabilidad para cada clase.</li>
</ol>

<p>Esto demuestra que el pipeline funciona de extremo a extremo, desde la carga hasta la predicción.</p>

<h2 style="color:#1ABC9C">Reproducibilidad y Dependencias</h2>

<p>Para garantizar la portabilidad y la ejecución en cualquier entorno, se generó un archivo <code>requirements.txt</code> con todas las dependencias del proyecto.</p>

<p>Este archivo puede utilizarse con:</p>

<pre><code>pip install -r requirements.txt</code></pre>

<p>Esto asegura que otros usuarios puedan replicar el entorno exacto y obtener los mismos resultados.</p>



In [32]:
# --------------------------------------------------------------------------------------------------------------
#                                         Orquestación
# --------------------------------------------------------------------------------------------------------------

# ------- 1. Instalar pytest -------
!pip install pytest > /dev/null

# ------- 2. Ejecutar las pruebas unitarias -------
!pytest test_processing.py

# ------- 3. Importar las funciones de los scripts creados -------
from train import run_training
from predict import make_prediction


# ------- Parte 1: Entrenamiento del modelo -------
metrics_result = run_training()
print("\n--- ENTRENAMIENTO COMPLETO ---")
print("Métricas del modelo:", metrics_result)

# ------- Parte 2: Predicción usando una fila del dataset original -------
import pandas as pd
import os
import kagglehub
from config import DATASET_URL, DATASET_FILE_NAME

print("\n--- PREDICCIÓN CON UNA MUESTRA DEL DATASET ORIGINAL ---")

try:
    # ------- Cargamos el dataset para poder seleccionar una fila de ejemplo -------
    path = kagglehub.dataset_download(DATASET_URL)
    csv_file_path = os.path.join(path, DATASET_FILE_NAME)
    df_original = pd.read_csv(csv_file_path, nrows=10000)

    # Seleccionamos la fila 3 (índice 2) como nuestra "muestra"
    # y la convertimos a un diccionario para que make_prediction la pueda procesar
    sample_data_from_df = df_original.iloc[2].to_dict()

    # ------- Realizamos la predicción con esta fila -------
    prediction_result = make_prediction(sample_data_from_df)

    print("\n--- PREDICCIÓN COMPLETA ---")
    print(prediction_result)

except FileNotFoundError as e:
    print(f"Error al cargar el archivo de datos original: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

# ------- 4. Generar y mostrar el archivo de requerimientos -------
print("\nGenerando archivo de requerimientos (requirements.txt)...")
!pip freeze > requirements.txt
print("\n--- Contenido de requirements.txt ---")
!cat requirements.txt

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.4.1, pluggy-1.6.0
rootdir: /content
plugins: anyio-4.9.0, langsmith-0.4.8, typeguard-4.4.4
collected 2 items                                                              

test_processing.py ..                                                    [100%]

============================== 2 passed in 1.81s ===============================

--- ENTRENAMIENTO COMPLETO ---
Métricas del modelo: {'accuracy': 0.8956, 'f1_score': 0.8792752601401571, 'precision': 0.8656902205576361, 'recall': 0.8956}

--- PREDICCIÓN CON UNA MUESTRA DEL DATASET ORIGINAL ---

--- PREDICCIÓN COMPLETA ---
{'prediction': 'Pago (Target=0)', 'probability_pago': np.float64(0.7997533881824829), 'probability_incumplimiento': np.float64(0.20024661181751713)}

Generando archivo de requerimientos (requirements.txt)...

--- Contenido de requirements.txt ---
absl-py==1.4.0
accelerate==1.9.0
aiofiles==24.1.0
ai

<div style="border: 2px solid #D5D8DC; padding: 20px; border-radius: 10px; background-color: #F8F9F9">

<h2 style="color:#2E4053">Conclusión</h2>

<p>
Este proyecto representó una simulación completa de un flujo de trabajo de <strong>Machine Learning en producción</strong>, estructurado bajo principios sólidos de ingeniería de software dentro de un entorno Jupyter/Colab.
</p>

<p>
Desde la configuración centralizada hasta la predicción de nuevas muestras, se implementaron prácticas reales como:
</p>

<ul>
  <li>Modularización del código mediante archivos virtuales con <code>%%writefile</code>.</li>
  <li>Preprocesamiento avanzado con <code>ColumnTransformer</code> y manejo de clases desbalanceadas con <code>SMOTETomek</code>.</li>
  <li>Entrenamiento y evaluación de un modelo robusto (<code>RandomForest</code>) con métricas clave.</li>
  <li>Predicción sobre datos reales utilizando el pipeline serializado.</li>
  <li>Pruebas unitarias, logging, manejo de errores y documentación con <code>type hints</code> y <code>docstrings</code>.</li>
  <li>Generación de <code>requirements.txt</code> para asegurar reproducibilidad.</li>
</ul>

<p>
Todo esto demuestra una comprensión profunda de cómo estructurar, documentar, probar y orquestar proyectos de ML en contextos reales. La elección del dataset permitió abordar un problema realista del sector financiero, con variables altamente heterogéneas, valores nulos, y desbalance significativo en la variable objetivo.
</p>

<p>
Este proyecto no solo responde a los requisitos académicos del curso, sino que además prepara una base sólida para futuros despliegues de modelos en entornos reales, como APIs o microservicios de predicción.
</p>

<p style="text-align: right; font-style: italic;">
— Adrián Ortega 😶
</p>

</div>
